Hi, I'm a beginner and now learning Variational Autoencoder with Pytorch, but it seems that it's not working well.

Here's a link to the discussion page!

https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/307169

What improvements are needed for successful learning?

for example,

change loss function speed up and train with more epochs (=> How can we speed up the process?) If the implementation of vae is successful, I think we can incorporate innovations such as recommending closer vectors in the latent space!

Any advice would be appreciated!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import FashionMNIST

import os
import cv2
import random
from PIL import Image
from tqdm import tqdm

# import torchbearer
# import torchbearer.callbacks as callbacks
# from torchbearer import Trial, state_key

# MU = state_key('mu')
# LOGVAR = state_key('logvar')

In [ ]:
def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    # Tensorflow
#     tf.random.set_seed(seed)

SEED = 42
fix_seed(SEED)

In [ ]:
trs_data = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
trs_data

In [ ]:
article_data = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
len(article_data)

In [ ]:
## 画像がない品物の情報を、article_dataから取り除く
for i in tqdm(range(105542)):
    base_path = '../input/h-and-m-personalized-fashion-recommendations/images'
    article_id = '0' + str(article_data.loc[i]['article_id']) + '.jpg'
    image_path = os.path.join(base_path, article_id[0:3], article_id)
    if (not os.path.exists(image_path)):
        article_data = article_data.drop(i)
        
    
len(article_data) #105100

In [ ]:
article_data = article_data.reset_index(drop=True)
article_data

In [ ]:
# ## 25782行目の画像は、(1,28,28)になっている
# for i in range(len(image_dataset)):
#     if (image_dataset[i].shape[0]!=3):
#         print(i)
    
#     if i%2000==0:
#         print("now{}".format(i))

In [ ]:
article_data = article_data.drop(25782)
article_data = article_data.reset_index(drop=True)
article_data

In [ ]:
def get_img_path(id):
    article_id = '0' + str(id) + '.jpg'
    base_path = "../input/h-and-m-personalized-fashion-recommendations/images"
    image_path = os.path.join(base_path, article_id[0:3], article_id)
    return image_path

article_data['path'] = article_data['article_id'].map(get_img_path)
img_path_list = article_data['path']
img_path_list[0]

In [ ]:
# import glob

# image_path_list = glob.glob("../input/h-and-m-personalized-fashion-recommendations/images/**/*.jpg", recursive=True)
# len(image_path_list)

# image_path_list[0]

In [ ]:
class fashion_Datasets(Dataset):
    def __init__(self, img_path_list, transform=None):
        self.img_path_list = img_path_list
        self.transform = transform
        
    def __len__(self):
        return len(self.img_path_list)
    
    def __getitem__(self, i):
        img = Image.open(self.img_path_list[i])
        
        if self.transform is not None:
            img = self.transform(img)
        
        return img

In [ ]:
transform =  transforms.Compose([transforms.Resize((28,28)), transforms.ToTensor()])
image_dataset = fashion_Datasets(img_path_list = img_path_list, transform=transform)

In [ ]:
# for i in range(len(image_dataset)):
#     if (image_dataset[i].shape[0]!=3):
#         print(i)
    
#     if i%2000==0:
#         print("now{}".format(i))

In [ ]:
# "0616100001.jpg"

In [ ]:
# article_data = article_data.drop(25782)
# article_data = article_data.reset_index(drop=True)
# article_data

In [ ]:
val_size = 500
train_size = 1000
others = len(image_dataset) - train_size - val_size
train_data, val_data, _ = torch.utils.data.random_split(image_dataset, [train_size, val_size, others])

In [ ]:
# for i in range(10000):
#     k = train_data[i]

In [ ]:
len(val_data)

In [ ]:
dataloader_train = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=2)
dataloader_valid = DataLoader(val_data, batch_size=64, shuffle=True, num_workers=2)

In [ ]:
# torch.log(0)によるnanを防ぐ
def torch_log(x):
    return torch.log(torch.clamp(x, min=1e-10))

In [ ]:
class VAE(nn.Module):
    def __init__(self, latent_size):
        super(VAE, self).__init__()
        self.latent_size = latent_size

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, 2, 1),  
            nn.ReLU(True),
            nn.Conv2d(32, 64, 3, 2, 1), 
            nn.ReLU(True),
            nn.Conv2d(64, 64, 3, 2, 1),  
            
        )
        
        self.mu = nn.Linear(64 * 4 * 4, latent_size)
        self.logvar = nn.Linear(64 * 4 * 4, latent_size)
        
        self.upsample = nn.Linear(latent_size, 64 * 3 * 3)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 64, kernel_size=3, stride=2),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 3, kernel_size=2, stride=2)  
        )

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
#             print("training")
            return eps.mul(std).add_(mu)
        else:
#             print("not training")
            return mu

    def forward(self, x):
        image = x
        x = self.encoder(x).relu().view(x.size(0), -1)
        
        mu = self.mu(x)
        logvar = self.logvar(x)
        z = self.reparameterize(mu, logvar)
        
        result = self.decoder(self.upsample(z).relu().view(-1, 64, 3, 3))
        
#         if state is not None:
#             state[torchbearer.Y_TRUE] = image
#             state[MU] = mu
#             state[LOGVAR] = logvar
        
        return result, z
    
    def loss(self, x):
        
#         print('x.shape')
#         print(x.shape)
#         print(self.encoder(x).relu().shape)
        out = self.encoder(x).relu().view(x.size(0), -1) #(64,3,28,28) -> (64,64,4,4) -> (64, 1024)
#         print('out.shape')
#         print(out.shape)
#         print('loss')
        mu = self.mu(out)
#         print('mu.shape')
#         print(mu.shape)
        log_var = self.logvar(out)
#         print('log_var.shape')
#         print(log_var.shape)
        
        KL = -0.5 * torch.mean(torch.sum(1 + log_var - mu**2 - torch.exp(log_var), dim=1))
        
        z = self.reparameterize(mu, log_var)
#         print(self.upsample(z).relu().shape)
#         print("upsample")
#         print(z.shape)
        y = self.decoder(self.upsample(z).relu().view(-1,64,3,3))
        y = y.view(-1, 3*28*28)
        x1 = x.view(-1, 3*28*28)
        
        reconstruction = torch.mean(torch.sum(x1 * torch_log(y) + (1-x1) * torch_log(1-y), dim=1))
        
        return KL, -reconstruction

In [ ]:
z_dim = 20
n_epochs = 40
lr = 0.001
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = VAE(z_dim).to(device)

In [ ]:
print(next(model.parameters()).is_cuda)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
n_epoch=5

In [ ]:
for epoch in range(n_epochs):
    if (epoch%15==0):
      lr /= 5
    losses = []
    KL_losses = []
    reconstruction_losses = []
    model.train()
    for x in tqdm(dataloader_train):
        x = x.to(device)

        model.zero_grad()

        y = model(x)

        KL_loss, reconstruction_loss = model.loss(x)

        loss = KL_loss + reconstruction_loss

        loss.backward()
        optimizer.step()

        losses.append(loss.cpu().detach().numpy())
        KL_losses.append(KL_loss.cpu().detach().numpy())
        reconstruction_losses.append(reconstruction_loss.cpu().detach().numpy())

    losses_val = []
    model.eval()
    for x in tqdm(dataloader_valid):

        x = x.to(device)

        y = model(x)

        KL_loss, reconstruction_loss = model.loss(x)

        loss = KL_loss + reconstruction_loss

        # WRITE ME

        losses_val.append(loss.cpu().detach().numpy())

    print('EPOCH:%d, Train Lower Bound:%lf, (%lf, %lf), Valid Lower Bound:%lf' %
          (epoch+1, np.average(losses), np.average(KL_losses), np.average(reconstruction_losses), np.average(losses_val)))

In [ ]:
model.eval()
for x in tqdm(dataloader_valid):

    x = x.to(device)

    y, z = model(x)

    break

In [ ]:
original_im = train_data[0].numpy()
original_im = np.transpose(original_im, (1,2,0))
plt.imshow(original_im)

In [ ]:
x = x.cpu().detach().numpy()
y = y.cpu().detach().numpy()

In [ ]:
x0 = x[18]
x0 = np.transpose(x0,(1,2,0))
plt.imshow(x0)

In [ ]:
y0 = y[18]
y0 = np.transpose(y0,(1,2,0))
plt.imshow(y0)

In [ ]:
x0 = x[54]
x0 = np.transpose(x0,(1,2,0))
plt.imshow(x0)

In [ ]:
y0 = y[54]
y0 = np.transpose(y0,(1,2,0))
plt.imshow(y0)

In [ ]:
x = train_data[0].numpy()

In [ ]:
img1 =cv2.imread("../input/h-and-m-personalized-fashion-recommendations/images/010/0108775015.jpg")
img = img1 / 256
img.shape

In [ ]:
np.transpose(img,(2,1,0)).shape

In [ ]:
img2 = train_data[1523].numpy()
img2 = np.transpose(img2, (1,2,0))
plt.imshow(img2)

In [ ]:
img = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
plt.imshow(img*256)

In [ ]:
plt.imshow(np.transpose(x, (1,2,0)))

In [ ]:
plt.imshow(np.transpose(x[12], (1,2,0)))

In [ ]:
img = np.transpose(result[3], (2,1,0))
plt.imshow(img)

In [ ]:
sample_img = cv2.imread('../input/h-and-m-personalized-fashion-recommendations/images/010/0108775015.jpg')
sample_img.shape

In [ ]:
plt.imshow(result[0])

In [ ]:
z = z.cpu().detach().numpy()

In [ ]:
from sklearn.neighbors import NearestNeighbors
extractor = NearestNeighbors(metric='euclidean', n_neighbors=4)
z[1:].shape